In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Suppose that we have a local constitutive law such that
$$
\varepsilon^{el}=M^{el}:\sigma
$$
for each individual grains.

In [2]:
inv=np.linalg.inv

-- Transformation rules for (2x1) matrix and (2x2) matrix.

In [3]:
def trans_2d_vec(a,r):
    b=np.zeros(2)
    for i in range(2):
        for j in range(2):
            b[i]=b[i]+r[i,j]*a[j]
    return b
def trans_2d_ten(a,r):
    b=np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    b[i,j]=b[i,j]+r[i,k]*r[j,l]*a[k,l]
    return b

In [4]:
ngr=3
weights=np.zeros(ngr)
weights[::]=1./ngr

In [5]:
thetas=[0,45,90]
thetas=np.array(thetas)*np.pi/180. ## to radian
R=np.zeros((2,2,len(thetas)))

for i in range(len(thetas)):
    r=np.zeros((2,2))
    r[0,0]=cos(thetas[i])
    r[0,1]=-sin(thetas[i])
    r[1,0]=sin(thetas[i])
    r[1,1]=cos(thetas[i])
    R[:,:,i]=r[:,:]

- Local elastic modulus

In [6]:
c=np.zeros((2,2))
c[0,0]=200000
c[1,1]=100000

In [7]:
print ('c:',c)

('c:', array([[200000.,      0.],
       [     0., 100000.]]))


## let's calculate the moduli in the lab axes coordinates

In [8]:
cgr=np.zeros((2,2,ngr))

In [9]:
for igr in range(ngr):
    r=R[:,:,igr]
    dum=trans_2d_ten(c,r)
    print('dum:',dum)
    cgr[:,:,igr]=dum

('dum:', array([[200000.,      0.],
       [     0., 100000.]]))
('dum:', array([[150000.,  50000.],
       [ 50000., 150000.]]))
('dum:', array([[1.000000e+05, 6.123234e-12],
       [6.123234e-12, 2.000000e+05]]))


In [10]:
print('cgr:',cgr.T)

('cgr:', array([[[2.000000e+05, 0.000000e+00],
        [0.000000e+00, 1.000000e+05]],

       [[1.500000e+05, 5.000000e+04],
        [5.000000e+04, 1.500000e+05]],

       [[1.000000e+05, 6.123234e-12],
        [6.123234e-12, 2.000000e+05]]]))


- weighted average of elastic stiffness

In [11]:
cav=np.zeros((2,2))
for igr in range(ngr):
    cav[:,:]=cav[:,:]+cgr[:,:,igr]*weights[igr]

In [12]:
print(cav/1000)

[[150.          16.66666667]
 [ 16.66666667 150.        ]]


In [13]:
sav1=inv(cav)
print sav1

[[ 6.75e-06 -7.50e-07]
 [-7.50e-07  6.75e-06]]


* Extreme case 2.

In [14]:
sgr=cgr.copy()
for igr in range(ngr):
    dum=cgr[:,:,igr]
    sgr[:,:,igr]=inv(dum)

In [15]:
sav=np.zeros((2,2))
for igr in range(ngr):
    sav[:,:]=sav[:,:]+sgr[:,:,igr]*weights[igr]

In [16]:
print sav*100*12

[[ 0.009 -0.001]
 [-0.001  0.009]]


In [17]:
sav2=sav[::]
print sav2

[[ 7.50000000e-06 -8.33333333e-07]
 [-8.33333333e-07  7.50000000e-06]]


** Let's load our polycrystal

In [18]:
sig=[100,200]

- Case 1

In [19]:
eps1=np.dot(sav1,sig)
print eps1

[0.000525 0.001275]


In [20]:
for i in range(2):
    dum=0
    for j in range(2):
        dum=dum+sav1[i,j]*sig[j]
    print dum

0.0005249999999999999
0.0012749999999999999


- Case 2

In [21]:
eps2=np.dot(sav2,sig)
print eps2

[0.00058333 0.00141667]


- Static condition - same stress for all grains as macroscopically given

In [22]:
sg=np.zeros((2,ngr))

In [23]:
for igr in range(ngr):
    sg[:,igr]=sig[::]

- Calculate epsilon el for each grain

In [24]:
sgr[:,:,igr]

array([[ 1.000000e-05, -3.061617e-22],
       [-3.061617e-22,  5.000000e-06]])

In [25]:
eel=np.zeros((2,ngr))
for igr in range(ngr):
    sx=sg[:,igr]/1000. # convert MPa GPa
    compliance=sgr[:,:,igr]
    print sx
    print compliance
    print '----'
    eel[:,igr]=np.dot(compliance,sx)

[0.1 0.2]
[[5.e-06 0.e+00]
 [0.e+00 1.e-05]]
----
[0.1 0.2]
[[ 7.5e-06 -2.5e-06]
 [-2.5e-06  7.5e-06]]
----
[0.1 0.2]
[[ 1.000000e-05 -3.061617e-22]
 [-3.061617e-22  5.000000e-06]]
----


## calculate weighted average of elastic strains

In [26]:
eav=np.zeros(2)
for igr in range(ngr):
    eav=eav+eel[:,igr]*weights[igr]

In [27]:
print 'The weighted average of elastic strain:',eav

The weighted average of elastic strain: [5.83333333e-07 1.41666667e-06]
